In [1]:
from gurobipy import *
import numpy as np

#set seed
np.random.seed(123)

#note: all i and j values are based on the array in question in x[i,j] format

In [2]:
#model
def model_setup():
    
    fish = Model("Fish")

    #variables
    x = fish.addVars(ARCS1, name = "Order") #order for fish i from supplier j
    y = fish.addVars(M, vtype = GRB.INTEGER, name = "Trucks") #trucks from supplier to warehouse
    l = fish.addVars(L, L, vtype=GRB.BINARY, name = "Location") #TSP for restaurants
    u = fish.addVars(L) #TSP u-variable

    #objective
    fish.setObjective(quicksum(cost[i,j]*x[i,j] for (i,j) in ARCS1) + #cost of procurement
                      quicksum(truckdel[i]*y[i] for i in range(M)) + #cost of trucks from supplier to warehouse
                      quicksum(travel[i,j]*l[i,j] for i in range(L) for j in range(L)), #cost of TSP from warehouse to restaurants
                      GRB.MINIMIZE) 

    #constraints
    fish.addConstrs((x[i,j] <= supply[i,j] for (i,j) in ARCS1), "Supply") #order <= supply
    fish.addConstrs((quicksum(x[i,j] for (i,j) in ARCS1.select('*',j)) >= demand2[j] for j in range(N)), "Demand") #total fish >= total demand
    fish.addConstrs((quicksum(x[i,j] for (i,j) in ARCS1.select(i,'*')) <= 50*y[i] for i in range(M)), "Trucks") #50 fish in each delivery truck
    
    fish.addConstrs((quicksum(l[i,j] for j in range(L)) == 1 for i in range(L)), "Outflow") #outflow nodes for TSP
    fish.addConstrs((quicksum(l[i,j] for i in range(L)) == 1 for j in range(L)), "Inflow") #inflow nodes for TSP
    
    fish.addConstrs((u[i] + 1 - u[j] <= 10000*(1 - l[i,j])  for i in range(L) for j in range(1,L) ), "Subtour") #subtour constraint for TSP
        #Big M: 10000
        
    precedent_pair = tuplelist()
    for i in range(L):
        for j in range(L):
            if j!=i:
                if i==0:
                    precedent_pair.append((i,j))
    fish.addConstrs((u[i] <= u[j] for (i,j) in precedent_pair), "Precedence") #warehouse must precede restaurants
    
    fish.setParam('OutputFlag', False)
    
    return fish

In [3]:
cost = np.array([[ 1.5,  2.5,  4.5,  0.0, 12.0],
                 [ 2.0,  2.6,  5.5,  6.0, 15.0],
                 [ 1.0,  0.0,  5.0,  7.0, 10.0],
                 [ 1.2,  2.0,  7.0,  8.0, 13.0],
                 [ 1.3,  2.2,  0.0, 10.0, 14.0]]) #cost of fish procurement

M, N = cost.shape #supply nodes: M, fish types: N

ARCS1 = tuplelist([(0,0), (0,1), (0,2), (0,3), (0,4),
                  (1,0), (1,1), (1,2), (1,3), (1,4),
                  (2,0), (2,1), (2,2), (2,3), (2,4),
                  (3,0), (3,1), (3,2), (3,3), (3,4),
                  (4,0), (4,1), (4,2), (4,3), (4,4)]) #arcs of suppliers' fish

truckdel = np.array([40, 80, 50, 70, 65]) #truck delivery cost from supplier to warehouse, contain 50 fish each

supply = np.array([[200, 250, 400,   0,  50], 
                   [240, 280, 300,  20, 100],
                   [180,   0, 250, 100,  60],
                   [150, 160, 220, 160, 160],
                   [300, 250,   0, 220,  60]]) #supply of fish j from supplier i

#demand
mean = np.array([[190, 100,  90,  25,  15],
                 [ 30, 120, 100,  30,  50],
                 [ 80,  50,  90,   5,  50],
                 [ 50, 160, 150,  20,   5],
                 [ 70,  40,  20, 150,  30]]) #demand of fish j from restaurant i

stdev = np.array([[30, 15, 20,  5,  8],
                  [20, 50, 30, 12,  8],
                  [15, 15, 12,  2,  5],
                  [ 8, 45, 30,  5,  3],
                  [15, 10,  3, 35,  8]]) #demand of fish j from restaurant i

#TSP
travel = np.array([[ 0, 51, 60, 42, 80, 66],
                   [51,  0, 27, 35, 42, 32],
                   [60, 27,  0, 40, 19, 25],
                   [42, 35, 40,  0, 64, 22],
                   [80, 42, 19, 64,  0, 57],
                   [66, 32, 25, 22, 57,  0]]) #TSP cost of travel from restaurant i to restaurant j

L = travel.shape[0] #location count: L

In [4]:
Sample_Size = 1000 #sample size of 1000
cost_full = np.zeros(Sample_Size)
allsol = []
alltrucks = []
allpermu = []

for k in range(Sample_Size):
    udemand = np.zeros(shape = (M, N))

    #loop to obtain demand samples
    for i in range(M):
        for j in range(N):
            meanval = mean[i][j]
            stdevval = stdev[i][j]
            demand = np.ceil(np.maximum(np.random.normal(meanval, stdevval), 0))
            udemand[i][j] = demand
    
    #asssuming common warehouse
    demand2 = udemand.sum(axis=0) #fish
    demand3 = udemand.sum(axis=1) #restaurants
    
    #solving
    fish = model_setup()

    fish.optimize()

    #optimals
    count = 0
    i, j = 0, 0
    sol = np.zeros(shape = (M+1, N))

    #obtain orders and trucks
    for v in fish.getVars():
        if ('Order' in v.VarName or 'Trucks' in v.VarName):
            sol[i, j] = v.x
            count += 1
            j += 1
            if count % 5 == 0:
                j = 0
                i += 1 
    trucks = sol[5]
    sol = sol[:-1]
    
    #obtain TSP solution
    s_edge = []
    temp = []
    for v in fish.getVars():
        if v.x > 0.001 and ('Location' in v.VarName):
            temp.append(v.VarName)
    for i in temp:
        path = tuple(list([int(i[-4]), int(i[-2])]))
        edge = np.add(path, (1,1))
        s_edge.append(edge)
    
    #obtain TSP permutation
    permu = np.zeros(L)
    predecessor = 1
    for i in range(L):
        for s in s_edge:
            if s[0] == predecessor:
                permu[i] = s[0]
                predecessor = s[1]
                break    
    
    #append all samples
    alltrucks.append(trucks)
    allsol.append(sol)
    allpermu.append(permu)
    cost_full[k] = fish.objVal

Academic license - for non-commercial use only - expires 2021-12-24
Using license file C:\Users\chant\gurobi.lic


In [5]:
finalsol = np.ceil((np.array(allsol)).mean(axis = 0)) #calculate mean orders
finaltrucks = np.ceil((np.array(alltrucks)).mean(axis = 0)) #calculate mean trucks required
finalpermu = (np.array(allpermu) - 1).mean(axis = 0) #calculate TSP permutation (actually same across all samples)

print("\n Optimal solution:")
for i in range(len(finalsol)):
    print("Supplier " + str(i))
    print(finalsol[i].tolist())

print("\n Trucks:", finaltrucks.tolist())

print("\n Permutation: ", finalpermu)
    
print("\n Minimal cost:", np.average(cost_full))


 Optimal solution:
Supplier 0
[196.0, 243.0, 396.0, 0.0, 50.0]
Supplier 1
[1.0, 13.0, 5.0, 20.0, 0.0]
Supplier 2
[180.0, 0.0, 51.0, 100.0, 60.0]
Supplier 3
[26.0, 150.0, 0.0, 112.0, 44.0]
Supplier 4
[23.0, 71.0, 0.0, 2.0, 0.0]

 Trucks: [18.0, 1.0, 8.0, 7.0, 2.0]

 Permutation:  [0. 3. 5. 2. 4. 1.]

 Minimal cost: 9152.117600000001
